## Setup

Note: not needed when reusable (database operations) code published on internal PyPI and pip installed into the env

In [1]:
import sys
from pathlib import Path

parent_path = str(Path.cwd())

if parent_path not in sys.path:
    sys.path.append(parent_path)

from db_operations.db import SQLite

## First of all, what tables and schemas can we find in the database?

also found here: https://github.com/elsirion/fedimint-observer/blob/master/schema/v0.sql

furthermore, data structures, for example federation config, can be found in https://github.com/fedimint/fedimint/tree/master/fedimint-core:

https://github.com/fedimint/fedimint/blob/374e2942bfcf67e9901cd410e7b5d3faeeae2c3b/fedimint-core/src/config.rs#L200

In [2]:
db_file = 'fedimint-observer.db'

with SQLite(db_file) as db:
    # Get a list of all tables
    tables = db.execute("SELECT name FROM sqlite_master WHERE type='table';")

    # Print table names
    print("Tables:")
    for table in tables:
      print(table[0])

    # Get schema for each table
    print("\nSchemas:")
    for table in tables:
      table_name = table[0]
      columns = db.execute(f"PRAGMA table_info('{table_name}');")

      print(f"\nTable: {table_name}")
      for column in columns:
        # print(column)
        column_id, column_name, column_type, not_null, default_value, pk = column
        print(f"\tColumn: {column_name}, Type: {column_type}, Nullable: {'NOT NULL' if not_null else 'NULL'}, Default: {default_value}, Primary Key: {'YES' if pk else 'NO'}")

Tables:
federations
sessions
transactions
transaction_inputs
transaction_outputs
ln_contracts
block_times
block_height_votes

Schemas:

Table: federations
	Column: federation_id, Type: BLOB, Nullable: NOT NULL, Default: None, Primary Key: YES
	Column: config, Type: BLOB, Nullable: NOT NULL, Default: None, Primary Key: NO

Table: sessions
	Column: federation_id, Type: BLOB, Nullable: NOT NULL, Default: None, Primary Key: YES
	Column: session_index, Type: INTEGER, Nullable: NOT NULL, Default: None, Primary Key: YES
	Column: session, Type: BLOB, Nullable: NOT NULL, Default: None, Primary Key: NO

Table: transactions
	Column: txid, Type: BLOB, Nullable: NOT NULL, Default: None, Primary Key: YES
	Column: federation_id, Type: BLOB, Nullable: NOT NULL, Default: None, Primary Key: YES
	Column: session_index, Type: INTEGER, Nullable: NOT NULL, Default: None, Primary Key: NO
	Column: item_index, Type: INTEGER, Nullable: NOT NULL, Default: None, Primary Key: NO
	Column: data, Type: BLOB, Nullable

## 1. How much Bitcoin was pegged-in to this federation?

In [3]:
import pandas as pd

with SQLite(db_file) as db:
    df_federations = pd.read_sql_query("SELECT federation_id, config FROM federations", db.conn)

In [4]:
# Confirm that this database only cotains data for one federation, should be "Bitcoin Principles"
df_federations['federation_id'].nunique() == 1

True

In [5]:
unique_value = df_federations['federation_id'].unique()[0]

In [6]:
with SQLite(db_file) as db:
    df_transaction_inputs= pd.read_sql_query("SELECT * FROM transaction_inputs", db.conn)

In [7]:
df_transaction_inputs

,federation_id,txid,in_index,kind,ln_contract_id,amount_msat
0,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\xfc\xea9rc\xa2>\xe2\x92\xdc\x06\xa7\x03;\xe...,0,wallet,None,1.000000e+10
1,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'%4\xe6\xb166\xaaE\xb0\x10\xcdk\xda6\xf3\xf2{...,0,mint,None,1.280000e+02
2,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'%4\xe6\xb166\xaaE\xb0\x10\xcdk\xda6\xf3\xf2{...,1,mint,None,5.120000e+02
3,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'%4\xe6\xb166\xaaE\xb0\x10\xcdk\xda6\xf3\xf2{...,2,mint,None,1.024000e+03
4,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'%4\xe6\xb166\xaaE\xb0\x10\xcdk\xda6\xf3\xf2{...,3,mint,None,4.096000e+03
...,...,...,...,...,...,...
692237,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\x19Ho\xb1\xf29k\x81\xeb$\xec\xc9~\xf3\x19\x...,8,mint,None,5.120000e+02
692238,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\x19Ho\xb1\xf29k\x81\xeb$\xec\xc9~\xf3\x19\x...,9,mint,None,1.024000e+03
692239,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\x19Ho\xb1\xf29k\x81\xeb$\xec\xc9~\xf3\x19\x...,10,mint,None,1.024000e+03
692240,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\x19Ho\xb1\xf29k\x81\xeb$\xec\xc9~\xf3\x19\x...,11,mint,None,1.024000e+03


In [8]:
df_transaction_inputs.kind.unique()

array(['wallet', 'mint', 'ln', 'stability_pool'], dtype=object)

In [9]:
import pandas as pd

def calculate_total_bitcoin_pegged_in_or_out(df, federation_id, peg_kinds):
    """
    Calculate the total amount of Bitcoin pegged-in or -out for a given federation.

    Parameters:
    - df: pandas DataFrame containing the transaction data.
    - federation_id: the federation_id to filter by (must match the BLOB format in the DataFrame).
    - peg_kinds: list of kind identifiers that correspond to peg-in transactions.

    Returns:
    - total_btc: Total amount of Bitcoin pegged-in or out for the given federation.
    """
    
    # Filter transaction inputs for the specific federation and peg-in kinds
    filtered_df = df[
        (df['federation_id'] == federation_id) &
        (df['kind'].isin(peg_kinds))
    ]

    # Calculate the total pegged-in Bitcoin in BTC
    total_msat = filtered_df['amount_msat'].sum()
    total_btc = total_msat / 100000000000.0  # Convert milli-satoshis to Bitcoin

    return total_btc


In [10]:
# Assuming that peg-in transactions are primarily associated with the wallet module of the Fedimint system
pegged_in = calculate_total_bitcoin_pegged_in_or_out(df_transaction_inputs, unique_value, ['wallet'])

In [11]:
pegged_in

np.float64(8.42375871)

## 2. How much Bitcoin was pegged-out from this federation?

In [12]:
with SQLite(db_file) as db:
    df_transaction_outputs= pd.read_sql_query("SELECT * FROM transaction_outputs", db.conn)

In [13]:
df_transaction_outputs

,federation_id,txid,out_index,kind,ln_contract_interaction_kind,ln_contract_id,amount_msat
0,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\xfc\xea9rc\xa2>\xe2\x92\xdc\x06\xa7\x03;\xe...,0,mint,None,None,1.0
1,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\xfc\xea9rc\xa2>\xe2\x92\xdc\x06\xa7\x03;\xe...,1,mint,None,None,1.0
2,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\xfc\xea9rc\xa2>\xe2\x92\xdc\x06\xa7\x03;\xe...,2,mint,None,None,2.0
3,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\xfc\xea9rc\xa2>\xe2\x92\xdc\x06\xa7\x03;\xe...,3,mint,None,None,2.0
4,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...",b'\xfc\xea9rc\xa2>\xe2\x92\xdc\x06\xa7\x03;\xe...,4,mint,None,None,2.0
...,...,...,...,...,...,...,...
769475,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...","b',q\xf1w\xcbe\x10\x15\x02\xc1\x95{\xe2mq\x89 ...",4,mint,None,None,128.0
769476,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...","b',q\xf1w\xcbe\x10\x15\x02\xc1\x95{\xe2mq\x89 ...",5,mint,None,None,128.0
769477,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...","b',q\xf1w\xcbe\x10\x15\x02\xc1\x95{\xe2mq\x89 ...",6,mint,None,None,256.0
769478,"b""\xb2\x10h\xc8O[\x12\xcaO\xdf\x93\xf3\xe4C\xd...","b',q\xf1w\xcbe\x10\x15\x02\xc1\x95{\xe2mq\x89 ...",7,mint,None,None,512.0


In [14]:
df_transaction_outputs.kind.unique()

array(['mint', 'ln', 'stability_pool', 'wallet'], dtype=object)

In [15]:
# Assuming that peg-out transactions are primarily associated with within the wallet module of the Fedimint system
pegged_out = calculate_total_bitcoin_pegged_in_or_out(df_transaction_outputs, unique_value, ['wallet'])

In [16]:
pegged_out

np.float64(7.81928988)

## 3. What is the current on-chain balance of this federation?

In [17]:
pegged_in - pegged_out

np.float64(0.6044688299999992)